In [65]:
#Importing dependencies and libraries
import numpy as np
import json
import requests
import pandas as pd

from config import api_key
from citipy import citipy

In [89]:
#Building the 500+ cities list

#setting ranges to look for cities
#randint(low[, high, size, dtype])

latitude_range = np.random.randint(-90,91, 70)
longitude_range = np.random.randint(-180,181, 70)

#nested for loop that will retrieve every city
cities_list = []

for x in longitude_range:
    for y in latitude_range:
        city = citipy.nearest_city(x,y)
        cities_list.append(city.city_name)

#converting the resulting list to a dataframe, dropping the duplicates and reindexing
cities_list_df = pd.DataFrame({'Cities':cities_list})
cities_list_df = cities_list_df.drop_duplicates().reset_index(drop = True)
cities_list_df

,Cities
0,bredasdorp
1,port elizabeth
2,ushuaia
3,hermanus
4,albany
...,...
687,corinto
688,gedo
689,tchaourou
690,kadambur


In [90]:
#API requests

# Lists initialization
latitude = []
longitude = []
temperature = []
humidity = []
cloudiness = []
wind = []

# Base URL
url = "http://api.openweathermap.org/data/2.5/weather?"

# for loop to request data from all cities
for city in cities_list_df['Cities']:
    
    # Build query URL
    query_url = url + "appid=" + api_key + "&q=" + city + "&units=imperial"
    
    # Request data for the current city
    weather_response = requests.get(query_url)
    weather_json = weather_response.json()
    
    #Making sure the request was succesful
    if weather_response.status_code == 200:
        
        # Appending current data to lists
        latitude.append(weather_json['coord']['lat'])
        longitude.append(weather_json['coord']['lon'])
        temperature.append(weather_json['main']['temp'])
        humidity.append(weather_json['main']['humidity'])
        cloudiness.append(weather_json['clouds']['all'])
        wind.append(weather_json['wind']['speed'])
    
    else:
        
        #print out the failed response
        print(f'The request for {city} failed with code {weather_response.status_code}')
    
    # Making sure that time between requests is enough so that API key doesn't get blocked
    sleep(1.01)

http://api.openweathermap.org/data/2.5/weather?appid=7e663d6c7e7fc4d8c6a4dc1906a66858&q=bredasdorp&units=imperial
http://api.openweathermap.org/data/2.5/weather?appid=7e663d6c7e7fc4d8c6a4dc1906a66858&q=port elizabeth&units=imperial
http://api.openweathermap.org/data/2.5/weather?appid=7e663d6c7e7fc4d8c6a4dc1906a66858&q=ushuaia&units=imperial
http://api.openweathermap.org/data/2.5/weather?appid=7e663d6c7e7fc4d8c6a4dc1906a66858&q=hermanus&units=imperial
http://api.openweathermap.org/data/2.5/weather?appid=7e663d6c7e7fc4d8c6a4dc1906a66858&q=albany&units=imperial
http://api.openweathermap.org/data/2.5/weather?appid=7e663d6c7e7fc4d8c6a4dc1906a66858&q=kruisfontein&units=imperial
http://api.openweathermap.org/data/2.5/weather?appid=7e663d6c7e7fc4d8c6a4dc1906a66858&q=port alfred&units=imperial
http://api.openweathermap.org/data/2.5/weather?appid=7e663d6c7e7fc4d8c6a4dc1906a66858&q=busselton&units=imperial
http://api.openweathermap.org/data/2.5/weather?appid=7e663d6c7e7fc4d8c6a4dc1906a66858&q=mou

400